In [1]:
import numpy as np
import requests
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np

In [6]:
# Todo: Construct Nodes based on meshblock from Police CSV and Census
# output to cs
edge_df = pd.read_csv("./output/meshblock_edges.csv",header=None)
print(edge_df.head())

        0       1
0  450100  450100
1  450100  450300
2  450100  451000
3  451000  451100
4  451000  451000


In [12]:
police_df = pd.read_csv("./datasets/Police_Table_Full Data_data.csv")
police_df.drop(labels=["Year Month (copy 2)","Territorial Authority","Occurrence Hour Of Day","Occurrence Day Of Week","Locn Type Division","Location Type","Area Unit","Year Month","ANZSOC Group","Table 1","ANZSOC Subdivision"," Month Year"],axis=1,inplace=True)
print(police_df.head())

                 ANZSOC Division  Victimisations  Meshblock  \
0  Acts Intended to Cause Injury               1     432800   
1  Acts Intended to Cause Injury               1    1221700   
2  Acts Intended to Cause Injury               1    2381600   
3  Acts Intended to Cause Injury               1    1081500   
4  Acts Intended to Cause Injury               1    1205012   

   Number of Records      Weapon  
0                  1  Not Stated  
1                  1  Not Stated  
2                  1  Not Stated  
3                  1  Not Stated  
4                  1  Not Stated  


In [26]:
source_nodes = edge_df[0].tolist()
target_nodes = edge_df[1].tolist()
unique_meshblock_nodes = np.unique(source_nodes + target_nodes)
print(unique_meshblock_nodes)

[ 450100  450300  450500  450600  450701  450901  451000  451100  451200
  480900  490701 4000137 4000138 4000139 4000140 4001321 4011331 4011333
 4011334]


In [51]:
def construct_meshblock_node(unique_node_list,df):
    nodes_output_df = pd.DataFrame()
    for meshblock_node in unique_node_list:
        crimes = df[df["Meshblock"]==meshblock_node]
        row = pd.DataFrame([meshblock_node],columns=["Meshblock_Id"])

        for crime_row in crimes.iterrows():
            crime = crime_row[1]
            division,division_victims ,division_records= crime["ANZSOC Division"],crime["Victimisations"], crime["Number of Records"]

            division_victims_key ,division_records_key,weapon_key= division+":"+"victims",division+":"+"records","has_weapon"

            if division_victims_key not in row.columns:
                row[division_victims_key] = 0

            if division_records_key not in row.columns:
                row[division_records_key] = 0

            if weapon_key not in row.columns:
                row[weapon_key] = False

            row[division_victims_key] +=division_victims
            row[division_records_key] +=division_records
            if crime["Weapon"] is not None or crime["Weapon"] != "Not Applicable":
                row[weapon_key] = True
        nodes_output_df = pd.concat([nodes_output_df,row])
    return nodes_output_df

In [55]:
meshblock_nodes_df = construct_meshblock_node(unique_meshblock_nodes,police_df)
meshblock_nodes_df = meshblock_nodes_df.fillna(0)
print(meshblock_nodes_df.head(1))
meshblock_nodes_df.to_csv("./output/meshblock_nodes.csv",header=True)

   Meshblock_Id  Theft and Related Offences:victims  \
0        450100                                78.0   

   Theft and Related Offences:records has_weapon  \
0                                75.0       True   

   Unlawful Entry With Intent/Burglary, Break and Enter:victims  \
0                                               13.0              

   Unlawful Entry With Intent/Burglary, Break and Enter:records  \
0                                               13.0              

   Acts Intended to Cause Injury:victims  \
0                                   19.0   

   Acts Intended to Cause Injury:records  \
0                                   17.0   

   Robbery, Extortion and Related Offences:victims  \
0                                              5.0   

   Robbery, Extortion and Related Offences:records  \
0                                              3.0   

   Sexual Assault and Related Offences:victims  \
0                                          2.0   

   Sexual Assault